In [1]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing import image 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from tensorflow.image import rgb_to_grayscale

Using TensorFlow backend.


## NN for Normal vs NonNormal

In [2]:
def normal_nonnormal(x): 
    if x == 'Normal': 
        return x 
    else: 
        return 'Non-Normal'

df = pd.read_csv('../CombinedImages/CombinedUpdated.csv')
na_fill = {'VirusCategory1': 'Normal'}
df = df.fillna(value = na_fill)

df.VirusCategory1 = df.VirusCategory1.map(normal_nonnormal)
df = df.join(pd.get_dummies(df.VirusCategory1.values, prefix = 'type'))
columns_include = ['Normal']
df = df[['ImagePath', 'VirusCategory1'] + [f'type_{i}' for i in columns_include]]


In [3]:
df.type_Normal.value_counts()

1    5841
0     636
Name: type_Normal, dtype: int64

In [8]:
X = df[['ImagePath', 'VirusCategory1']]
y = df[[f'type_{i}' for i in columns_include]]

x_train, x_test, y_train, y_test = train_test_split(X,y, random_state = 10, stratify = X.VirusCategory1.values,
                                                   train_size = .95)

x_train = x_train.drop('VirusCategory1', axis = 1)
x_test = x_test.drop('VirusCategory1', axis = 1)

In [9]:
#print(len(df[df['type_Normal']==1]), len(df[df['type_Normal']==0]))

In [10]:
def get_image_value(path, img_type): 
    img = image.load_img(path, target_size = (28,28,3))
    img = image.img_to_array(img)
    if img_type == 'grey':
        img = np.dot(img[...,:3], [.2989,.5870,.1140])
    return img/255


def get_data(df, img_type): 
    from tqdm import tqdm
    img_list = [] 
    for path in tqdm(df.ImagePath.values):
        path = f'../CombinedImages/all/{path}'
        img_list.append(get_image_value(path, img_type)) 
    return np.array(img_list).squeeze()
img_type = 'normal'
x_test = get_data(x_test, img_type)
x_train = get_data(x_train, img_type)


100%|██████████| 6153/6153 [01:28<00:00, 69.69it/s]


In [ ]:
#plt.imshow(x_test[0], cmap = plt.get_cmap('gray'))

In [12]:
from imblearn.over_sampling import SMOTE

smote = SMOTE() 

x_train_resampled, y_train_resampled = smote.fit_sample(x_train, y_train)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [ ]:
import pickle 
pickle.dump(x_train, open('picklesNorm/SMOTE_x_train_normal.p', 'wb'), protocol = 2)
#pickle.dump(x_test, open('picklesNorm/Binary_x_test_normal.p', 'wb'))

# pickle.dump(x_train, open('picklesNorm/x_train_grey.p', 'wb'))
# pickle.dump(x_test, open('picklesNorm/x_test_grey.p', 'wb'))

pickle.dump(y_train, open('picklesNorm/SMOTE_y_train.p', 'wb'))
#pickle.dump(y_test, open('picklesNorm/Binary_y_test.p', 'wb'))

In [ ]:
assert False

In [ ]:
def get_image_value(path, img_type): 
    img = image.load_img(path, target_size = (28,28,3))
    img = image.img_to_array(img)
    if img_type == 'grey':
        img = np.dot(img[...,:3], [.2989,.5870,.1140])
    return img/255

img_type = 'normal'


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D 
from keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing import image 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from tensorflow.image import rgb_to_grayscale
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
def get_conv_model_normal(img_type):
    if img_type == 'normal': 
        inp_shape = (28,28,3)
    else: 
        inp_shape = (28,28, 1)
    act = 'sigmoid'
    drop = .25 
    kernal_reg = regularizers.l2(.001)
    dil_rate = 2
    adam = Adam(.001)
    
    model = Sequential() 
    
    model.add(Conv2D(128, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg, kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, dilation_rate = dil_rate, 
                     kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
   
    model.add(Flatten())
    model.add(Dropout(drop))

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(drop))

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model 
# def get_conv_model_normal(img_type):
#     if img_type == 'normal': 
#         inp_shape = (28,28,3)
#     else: 
#         inp_shape = (28,28, 1)
#     drop = .25 
#     kernal_reg = regularizers.l2(.001)
#     dil_rate = 2
#     model = Sequential() 
    
#     model.add(Conv2D(128, kernel_size=(3,3),activation='sigmoid', input_shape = inp_shape, 
#                      kernel_regularizer = kernal_reg, kernel_initializer = 'he_uniform', padding = 'same'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))


#     model.add(Conv2D(128, (1, 1), activation='sigmoid', kernel_regularizer = kernal_reg, dilation_rate = dil_rate, 
#                      kernel_initializer = 'he_uniform', padding = 'same'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    
# #     model.add(Conv2D(128, (1, 1), activation='relu', kernel_regularizer = kernal_reg, dilation_rate = dil_rate))
# #     model.add(MaxPooling2D(pool_size=(2, 2)))
    
#     model.add(Flatten())
#     model.add(Dropout(drop))
    
#     model.add(Dense(512, activation='relu'))
#     model.add(Dropout(drop))

#     model.add(Dense(2, activation='sigmoid'))
    
#     model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     return model 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


def get_samples_norm(img_type): 
    x_train = pickle.load(open(f'picklesNorm/Binary_x_train_{img_type}.p', 'rb'))
    x_test = pickle.load(open(f'picklesNorm/Binary_x_test_{img_type}.p', 'rb'))
    y_train = pickle.load(open(f'picklesNorm/Binary_y_train.p', 'rb'))
    y_test = pickle.load(open(f'picklesNorm/Binary_y_test.p', 'rb'))
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_samples_norm(img_type)
print(x_train.shape)


if img_type == 'grey': 
    x_train = x_train.reshape(len(x_train), 28, 28, 1)
    x_test = x_test.reshape(len(x_test), 28, 28, 1)
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=5)
model_checkpoint = ModelCheckpoint('models/Tester-NormalModelCheckpointWeights.h5', verbose = 1, save_best_only=True)

epochs = 200
batch_size = 32
normal_model = get_conv_model_normal(img_type)
normal_history = normal_model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, 
         callbacks = [early_stopping, model_checkpoint], validation_data = (x_test, y_test), verbose= 1)

In [ ]:
train_loss = normal_history.history['loss']
train_acc = normal_history.history['accuracy']
test_loss = normal_history.history['val_loss']
test_acc = normal_history.history['val_accuracy']
epochs = [i for i in range(1, len(test_acc)+1)]

fig, ax = plt.subplots(1,2, figsize = (15,5))
ax[0].plot(epochs, train_loss, label = 'Train Loss')
ax[0].plot(epochs, test_loss, label = 'Test Loss')
ax[0].set_title('Train/Test Loss')
ax[0].legend()

ax[1].plot(epochs, train_acc, label = 'Train Accuracy')
ax[1].plot(epochs, test_acc, label = 'Test Accuracy')
ax[1].set_title('Train/Test Accuracy')
ax[1].legend()

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
        # Create the basic matrix
#plt.figure(figsize = (10,10))

    plt.imshow(cm,  cmap=plt.cm.Blues) 
    # Add title and axis labels
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    # Add appropriate axis scales
    class_names = classes # Get class labels to add to matrix
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Add labels to each cell
    thresh = cm.max() / 2. # Used for text coloring below
    # Here we iterate through the confusion matrix and append labels to our visualization 
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment='center',
                     color='white' if cm[i, j] > thresh else 'black')

    # Add a legend
    plt.colorbar()
    plt.show()
    
normal_model = get_conv_model_normal(img_type)
normal_model.load_weights('models/Tester-NormalModelCheckpointWeights.h5')

y_test_predict = normal_model.predict(x_test).ravel()
# print(y_test_predict) 
y_test_predict = np.where(y_test_predict >= .5, y_test_predict, 0)
y_test_predict = np.where(y_test_predict <= .5, y_test_predict, 1)

#y_test_predict = y_test_predict[y_test_predict > .5] =1 
plot_confusion_matrix(confusion_matrix(y_test, y_test_predict), classes = [0,1])

In [ ]:
from sklearn.metrics import roc_curve, auc

# y_score = normal_model.

In [ ]:
normal_model = get_conv_model_normal(img_type)
normal_model.load_weights('models/Tester-NormalModelCheckpointWeights.h5')
tester_img = get_image_value('TestImages/Normal6.jpg', img_type)
if img_type == 'grey': 
    tester_img = np.reshape(tester_img, (1,28,28,1))
else:
    tester_img = np.reshape(tester_img, (1,28,28,3))
normal_predict = normal_model.predict(tester_img).squeeze()
print(normal_predict)
if normal_predict < .5: 
    print('Non-Normal')
else: 
    print('Normal')



In [ ]:
assert False

### Test on Random Images

['Normal', 'COVID-19', 'Bacterial', 'Fungal', 'SARS']
['Normal', 'COVID-19', 'Bacterial']